<a href="https://colab.research.google.com/github/jasonlee87/AIDL-Lab/blob/master/Copy_of_Copy_of_Copy_of_solaris_tf15_base_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2.1 Install_check

In [0]:
# 사용자가 버전 지정하지 않으면 자동 디폴트 버전이 로드된다.
import tensorflow as tf
print(tf.__version__)
# 출력: 1.15.0

# 문자열 출력 예
hello = tf.constant('Hello, TensorFlow 1.5')
sess = tf.Session()
print(sess.run(hello)) 
# 출력: b'Hello, TensorFlow 1.5'

# 더하기 연산 예
a = tf.constant(11)
b = tf.constant(22)
print(sess.run(a + b))
# 출력: 33

# 세션 닫기
sess.close()

1.15.0
b'Hello, TensorFlow 1.5'
33


3.1 Graph example

In [0]:
import tensorflow as tf

# 노드(node)를 생성하고 출력해본다
node1 = tf.constant(1.0, dtype=tf.float32)
node2 = tf.constant(2.0) # 생략시 tf.float32 타입으로 선언
print(node1) 
# 출력: Tensor("Const_3:0", shape=(), dtype=float32)
print(node2) 
# 출력: Tensor("Const_4:0", shape=(), dtype=float32)

# 세션을 열고 그래프를 실행합니다
sess = tf.Session()
print(sess.run([node1, node2])) 
# 출력: [1.0, 2.0]

# 2개의 노드를 더하는 노드3을 선언한다
node3 = tf.add(node1, node2)
print(node3) 
# 출력: Tensor("Add_1:0", shape=(), dtype=float32)

# 노드3 실행
print(sess.run(node3)) 
# 출력: 3.0

sess.close()

Tensor("Const_3:0", shape=(), dtype=float32)
Tensor("Const_4:0", shape=(), dtype=float32)
[1.0, 2.0]
Tensor("Add_1:0", shape=(), dtype=float32)
3.0


3.2 Placeholder

In [0]:
import tensorflow as tf

# 플레이스홀더 노드와 add 노드를 선언
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  
# 위 식은 암시적으로 tf.add(a, b) 형태로 간주됩니다.

# 세션을 열고 그래프를 실행합니다.
sess = tf.Session()
print(sess.run(adder_node, feed_dict={a: 1.5, b: 2.0}))
# 출력: 3.5
print(sess.run(adder_node, feed_dict={a: [1, 3], b: [2, 4]}))
# 출력: [3. 7.]

# 노드 추가해서 더 복잡한 그래프를 만들수있다
add_and_triple = adder_node * 3	
# 위 식은 암시적으로 tf.multiply(adder_node,3) 형태로 간주되니다
print(sess.run(add_and_triple, feed_dict={a: 1.5, b: 2.0}))
# 출력 : 10.5

sess.close()

3.5
[3. 7.]
10.5


3.3 Linear regression

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# 선형회귀 모델(Wx + b)을 정의합니다.
W = tf.Variable(tf.random_normal(shape=[1]))   
b = tf.Variable(tf.random_normal(shape=[1]))
x = tf.placeholder(tf.float32)
linear_model = W*x + b

# True Value를 입력받기위한 플레이스홀더를 정의합니다.
y = tf.placeholder(tf.float32)

# 손실 함수를 정의합니다.
loss = tf.reduce_mean(tf.square(linear_model - y)) # MSE 손실함수 \mean{(y' - y)^2}

# 최적화를 위한 그라디언트 디센트 옵티마이저를 정의합니다.
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

# 트레이닝을 위한 입력값과 출력값을 준비합니다. 
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]

# 세션을 실행하고 파라미터(W,b)를 noraml distirubtion에서 추출한 임의의 값으로 초기화합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 경사하강법을 1000번 수행합니다.
for i in range(1000):
  sess.run(train_step, feed_dict={x: x_train, y: y_train})

# 테스트를 위한 입력값을 준비합니다.
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형회귀 모델이 데이터의 경향성(y=2x)을 잘 학습했는지 측정합니다.
# 예상되는 참값 : [7, 10, 11, 12]
print(sess.run(linear_model, feed_dict={x: x_test}))

sess.close()

[ 6.991687  9.969417 10.961993 11.95457 ]


3.4 Linear regression with tensorboard

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# 선형회귀 모델(Wx + b)을 정의합니다.
W = tf.Variable(tf.random_normal(shape=[1]), name="W")   
b = tf.Variable(tf.random_normal(shape=[1]), name="b")
x = tf.placeholder(tf.float32, name="x")
linear_model = W*x + b

# True Value를 입력받기위한 플레이스홀더를 정의합니다.
y = tf.placeholder(tf.float32, name="y")

# 손실 함수를 정의합니다.
loss = tf.reduce_mean(tf.square(linear_model - y)) # MSE 손실함수 \mean{(y' - y)^2}
# 텐서보드를 위한 요약정보(scalar)를 정의합니다.
tf.summary.scalar('loss', loss)

# 최적화를 위한 옵티마이저를 정의합니다.
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)

# 트레이닝을 위한 입력값과 출력값을 준비합니다. 
x_train = [1, 2, 3, 4]
y_train = [2, 4, 6, 8]

# 세션을 실행하고 파라미터(W,b)를 noraml distirubtion에서 추출한 임의의 값으로 초기화합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 텐서보드 요약정보들을 하나로 합칩니다.
merged = tf.summary.merge_all()
# 텐서보드 summary 정보들을 저장할 폴더 경로를 설정합니다.
tensorboard_writer = tf.summary.FileWriter('./tensorboard_log', sess.graph)

# 경사하강법을 1000번 수행합니다.
for i in range(1000):
  sess.run(train_step, feed_dict={x: x_train, y: y_train})

  # 매스텝마다 텐서보드 요약정보값들을 계산해서 지정된 경로('./tensorboard_log')에 저장합니다.
  summary = sess.run(merged, feed_dict={x: x_train, y: y_train})
  tensorboard_writer.add_summary(summary, i)

# 테스트를 위한 입력값을 준비합니다.
x_test = [3.5, 5, 5.5, 6]
# 테스트 데이터를 이용해 학습된 선형회귀 모델이 데이터의 경향성(y=2x)을 잘 학습했는지 측정합니다.
# 예상되는 참값 : [7, 10, 11, 12]
print(sess.run(linear_model, feed_dict={x: x_test}))

sess.close()

[ 6.98079   9.929326 10.91217  11.895016]


4.1 MNIST classificatoin using softmax regression

In [0]:
# -*- coding: utf-8 -*-

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# 입력값과 출력값을 받기 위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

# 변수들을 설정하고 Softmax Regression 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

# cross-entropy 손실 함수와 옵티마이저를 정의합니다.
loss = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)) # tf.nn.softmax_cross_entropy_with_logits API를 이용한 구현
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 1000번 반복을 수행하면서 최적화를 수행합니다.
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})

# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("정확도(Accuracy): %f" % sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})) # 정확도 : 약 91%

sess.close()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

4.2 tf_nn_sparse_softmax_cross_entropy_with_logits_example

In [0]:
# -*- coding: utf-8 -*-
# tf.nn.sparse_softmax_cross_entropy_with_logits API를 사용한 Softmax Regression을 이용한 MNIST 숫자 분류

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False) # one_hot=False로 불러옵니다.

# 입력값과 출력값을 받기 위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.int64, shape=[None]) # tf.int64 타입으로 선언합니다.

# 변수들을 설정하고 Softmax Regression 모델을 정의합니다.
W = tf.Variable(tf.zeros(shape=[784, 10]))
b = tf.Variable(tf.zeros(shape=[10]))
logits = tf.matmul(x, W) + b
y_pred = tf.nn.softmax(logits)

# cross-entropy 손실 함수와 옵티마이저를 정의합니다.
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)) # tf.nn.softmax_cross_entropy_with_logits API를 이용한 구현
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)) # tf.nn.sparse_softmax_cross_entropy_with_logits API를 이용한 구현
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# 세션을 열고 변수들에 초기값을 할당합니다.
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# 1000번 반복을 수행하면서 최적화를 수행합니다.
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})

# 학습이 끝나면 학습된 모델의 정확도를 출력합니다.
correct_prediction = tf.equal(tf.argmax(y_pred,1), y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("정확도(Accuracy): %f" % sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})) # 정확도 : 약 91%

sess.close()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
정확도(Accuracy): 0.915000


5.1 MNIST classification using ann

In [0]:
# -*- coding: utf-8 -*-
# 텐서플로우를 이용한 ANN(Artificial Neural Networks) 구현

import tensorflow as tf

# MNIST 데이터를 다운로드 합니다.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# 학습을 위한 설정값들을 정의합니다.
learning_rate = 0.001
num_epochs = 30     # 학습횟수
batch_size = 256    # 배치개수
display_step = 1    # 손실함수 출력 주기
input_size = 784    # 28 * 28
hidden1_size = 256
hidden2_size = 256
output_size = 10

# 입력값과 출력값을 받기 위한 플레이스홀더를 정의합니다.
x = tf.placeholder(tf.float32, shape=[None, input_size])
y = tf.placeholder(tf.float32, shape=[None, output_size])

# ANN 모델을 정의합니다.
def build_ANN(x):
  W1 = tf.Variable(tf.random_normal(shape=[input_size, hidden1_size]))
  b1 = tf.Variable(tf.random_normal(shape=[hidden1_size]))
  H1_output = tf.nn.relu(tf.matmul(x,W1) + b1)
  W2 = tf.Variable(tf.random_normal(shape=[hidden1_size, hidden2_size]))
  b2 = tf.Variable(tf.random_normal(shape=[hidden2_size]))
  H2_output = tf.nn.relu(tf.matmul(H1_output,W2) + b2)
  W_output = tf.Variable(tf.random_normal(shape=[hidden2_size, output_size]))
  b_output = tf.Variable(tf.random_normal(shape=[output_size]))
  logits = tf.matmul(H2_output,W_output) + b_output

  return logits

# ANN 모델을 선언합니다.
predicted_value = build_ANN(x)

# 손실함수와 옵티마이저를 정의합니다.
# tf.nn.softmax_cross_entropy_with_logits 함수를 이용하여 활성함수를 적용하지 않은 output layer의 결과값(logits)에 softmax 함수를 적용합니다.
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predicted_value, labels=y))
train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 세션을 열고 그래프를 실행합니다.
with tf.Session() as sess:
  # 변수들에 초기값을 할당합니다.
  sess.run(tf.global_variables_initializer())

  # 지정된 횟수만큼 최적화를 수행합니다.
  for epoch in range(num_epochs):
    average_loss = 0.
    # 전체 배치를 불러옵니다.
    total_batch = int(mnist.train.num_examples/batch_size)
    # 모든 배치들에 대해서 최적화를 수행합니다.
    for i in range(total_batch):
      batch_x, batch_y = mnist.train.next_batch(batch_size)
      # 옵티마이저를 실행해서 파라마터들을 업데이트합니다.
      _, current_loss = sess.run([train_step, loss], feed_dict={x: batch_x, y: batch_y})
      # 평균 손실을 측정합니다.
      average_loss += current_loss / total_batch
    # 지정된 epoch마다 학습결과를 출력합니다.
    if epoch % display_step == 0:
      print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), average_loss))

  # 테스트 데이터를 이용해서 학습된 모델이 얼마나 정확한지 정확도를 출력합니다.
  correct_prediction = tf.equal(tf.argmax(predicted_value, 1), tf.argmax(y, 1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  print("정확도(Accuracy): %f" % (accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))) # 정확도: 약 94%

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

반복(Epoch): 1, 손실 함수(Loss): 244.681433
반복(Epoch): 2, 손실 함수(Loss): 61.558458
반복(Epoch): 3, 손실 함수(Loss): 41.715605
반복(Epoch): 4, 손실 함수(Loss): 31.207401
반복(Epoch): 5, 손실 함수(Loss): 24.420273
반복(Epoch): 6, 손실 함수(Loss): 19.678957
반복(Epoch): 7, 손실 함수(Loss): 16.224038
반복(Epoch): 8, 손실 함수(Loss): 13.498711
반복(Epoch): 9, 손실 함수(Loss): 11.005475
반복(Epoch): 10, 손실 함수(Loss): 9.384741
반복(Epoch): 11, 손실 함수(Loss): 7.626551
반복(Epoch): 12, 손실 함수(Loss): 6.375052
반복(Epoch): 13, 손실 함수(Loss): 5.380331
반복(Epoch): 14, 손실 함수(Loss): 4.392755
반복(Epoch): 15, 손실 함수(Loss): 3.637121
반복(Epoch): 16, 손실 함수(Loss): 3.048263
반복(Epoch): 17, 손실 함수(Loss)